<a href="https://colab.research.google.com/github/HuanAII/ResNet_CIFAR-10/blob/main/Resnet_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
class ResidualBloc(nn.Module):
    def __init__(self, in_planes, out_planes, stride):
        super().__init__()
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_planes)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != out_planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        return F.relu(out)


In [4]:
# ResNet-20 cho CIFAR-10
class ResNet20(nn.Module):
    def __init__(self, block= ResidualBloc, num_blocks=[3,3,3], num_classes=10):
        super(ResNet20, self).__init__()
        self.in_planes = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.linear = nn.Linear(64, num_classes)

    def _make_layer(self, block, out_planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for s in strides:
            layers.append(block(self.in_planes, out_planes, s))
            self.in_planes = out_planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)  # 32x32
        out = self.layer2(out)  # 16x16
        out = self.layer3(out)  # 8x8
        out = self.avgpool(out)  # 1x1
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [5]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# data processinh
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True,
                                        transform=transform_train)
trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True,
                                       transform=transform_test)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)


100%|██████████| 170M/170M [00:14<00:00, 11.7MB/s]


In [6]:
import torch.optim as optim

device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = ResNet20().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100, 150], gamma=0.1)

# Huấn luyện
for epoch in range(1, 30):
    net.train()
    running_loss = 0
    for inputs, targets in trainloader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    scheduler.step()
    print(f"Epoch {epoch}, Loss: {running_loss:.4f}")


Epoch 1, Loss: 654.8669
Epoch 2, Loss: 467.6530
Epoch 3, Loss: 351.6413
Epoch 4, Loss: 304.7737
Epoch 5, Loss: 278.3147
Epoch 6, Loss: 260.8681
Epoch 7, Loss: 249.1437
Epoch 8, Loss: 242.3949
Epoch 9, Loss: 234.2127
Epoch 10, Loss: 228.1078
Epoch 11, Loss: 220.3091
Epoch 12, Loss: 215.6382
Epoch 13, Loss: 214.6277
Epoch 14, Loss: 210.7878
Epoch 15, Loss: 208.7773
Epoch 16, Loss: 208.1936
Epoch 17, Loss: 202.3052
Epoch 18, Loss: 203.0416
Epoch 19, Loss: 199.8583
Epoch 20, Loss: 197.2634
Epoch 21, Loss: 197.5532
Epoch 22, Loss: 197.2686
Epoch 23, Loss: 192.2971
Epoch 24, Loss: 191.6381
Epoch 25, Loss: 190.6552
Epoch 26, Loss: 192.8948
Epoch 27, Loss: 190.0158
Epoch 28, Loss: 188.1512
Epoch 29, Loss: 188.2923


In [7]:
def evaluate(net, loader):
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return correct / total

acc = evaluate(net, testloader)
print(f"Accuracy on test set: {acc*100:.2f}%")


Accuracy on test set: 73.20%
